## LDA Preprocessing 1
# Artifact Removal and Stopword Selection
This notebook takes the clean JSON files for each article and does some preprocessing to obtain a text that we can analyze using LDA.

Specifically, we do:
* Artifact removal
* Stopword selection

In the next notebook we will do:
* Punctuation removal
* Lemmatization

In [1]:
import json
import re

We will also use some utility functions we defined in the `utils/` folder:
* `loadCorpusList(path)`: Loads the corpus as a list of `Article` objects (see `utils/Article.py`). This will allow us to save the clean text per document into the same JSON file with the metadata included.
* `saveCorpus(path)`: Saves the articles in JSON format in their current state. Useful when we want to append information to our clean JSON files.

In [2]:
import os
import sys 

# Jupyter Notebooks are not good at handling relative imports.
# Best solution (not great practice) is to add the project's path
# to the module loading paths of sys.

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.loaders import loadCorpusList, saveCorpus

## Loading the corpus

In [3]:
corpusPath = '../data/parsedHTML'

corpusList = loadCorpusList(corpusPath)

In [4]:
corpusPath = '../data/parsedPDF'

corpusList += loadCorpusList(corpusPath)

In [5]:
corpusList[0].title

'Kasahara, Javier. “Una relectura leibniziana al mecanicismo.”'

We will only work with documents in Spanish. Hence, let's replace `corpusList` with only the articles that are in Spanish.

In [6]:
corpusList = [doc for doc in corpusList if doc.lang == 'es']

This leaves is with approximately 1400 articles.

In [7]:
len(corpusList) 

1495

Let's check for duplicates in case we have some. This is important in case we downloaded an article twice, once in PDF and once in HTML.

In [8]:
len(set([doc.id for doc in corpusList]))

1495

## Artifact removal

There are some artifacts included in the text that are produced by HTML processing (or in the future because of how PDF files store text).

Let's start by removing numbers and some special characters such as newline characters (`\n`). We will keep normal punctuation for now as that might help SpaCy when we do lemmatization.

In [9]:
for doc in corpusList:
    doc.cleanText = re.sub('\d|\n',' ', doc.text)

We can detect some of these artifacts by looking for non-alphanumeric characters between alphanumeric characters (e.g. `"ar-gument"`, `"ar\xadgument"`).

In [10]:
artifacts = re.compile('\w+[^a-zA-ZáéíóúÁÉÍÓÚñÑüÜ\d\s:]\w+')

In [11]:
[re.findall(artifacts, doc.text) for doc in corpusList][0]

['35-48',
 'mecanicismo,1',
 'mente-cuerpo',
 'otro.2',
 'primero.3',
 '148-149',
 '125-194',
 'montenegrop.nicolas',
 'urosario.edu']

One common artifact is the hex `\xad` for the soft hyphen which is used to break lines. We can remove it easily.

In [12]:
for doc in corpusList:
    doc.cleanText = re.sub('\\xad','', doc.cleanText)
    doc.cleanText = doc.cleanText.replace(u"\xa0", "")

We can save the corpus for now.

In [13]:
if not os.path.exists('../data/corpus'):
    os.mkdir('../data/corpus')
saveCorpus('../data/corpus', corpusList)

## Stopword Removal
Stopword removal is perhaps the most difficult part of preprocessing. There are two challenges to meet:
* Some stopword lists such as the one included in NLTK for Spanish are too weak and do not filter many stopwords.
* Other stopword lists are too inclusive and can eliminate words that are meaningful in philosophy (e.g. 'verdadero', true). 
It is important to note that stopwords are very context-sensitive. A word in one context may provide little meaning (hence counting as a stopword) while in other contexts it may provide lots of information.

To tackle these challenges, we will first to an initial filtering with NTLK's list. This will leave many stopwords in the text, but will reduce the size of each text considerably. Then we will compare the text with a stronger list of stopwords (source). We will see which words are both the text and the stronger stopwords list. We will inspect these lists manually and extract a list of protected words. We will iterate over this process a number of times. Once we have a robust list of protected words, we will concatenate NLTK's stopwords list with the stronger one and eliminate the protected words from it. This will provide a final (hopefully middle ground) stopword list with which to continue.

In [14]:
from nltk.corpus import stopwords as nltk_stopwords

stopwords_weak = nltk_stopwords.words('spanish')

In [15]:
import requests

r = requests.get('https://raw.githubusercontent.com/stopwords-iso/stopwords-es/master/stopwords-es.txt')
stopwords_strong = r.text.split('\n')

In [16]:
docWords = []
for doc in corpusList:
    docWords += [word for word in re.findall('\w+', doc.cleanText) if word not in stopwords_weak]

In [17]:
docWords[:10]

['Documento',
 'título',
 'Kasahara',
 'Javier',
 'Una',
 'relectura',
 'leibniziana',
 'mecanicismo',
 'Eidos',
 'Kasahara']

In [18]:
from collections import Counter

docs_and_stopwords = Counter([word for word in docWords if word in stopwords_strong])

In [19]:
docs_and_stopwords.most_common(10)

[('u', 23849),
 ('ser', 23800),
 ('puede', 20530),
 ('si', 16871),
 ('sino', 12907),
 ('mismo', 12535),
 ('i', 10742),
 ('bien', 10412),
 ('decir', 9864),
 ('manera', 8405)]

Already in the first 10 most common words in both the documents and the strong list of stopwords we find words that in philosophy are quite meaningful:
* 'ser': being
* 'bien': good
* 'modo': mode

We will start saving those words and eliminating them from the stronger list of stopwords. Then we will repeat the process of selecting the words that are in both lists and see which words are common. By iterating over this process a couple of times, we will get a list of protected words.

In [20]:
protectedWords = [
    'ser',
    'bien',
    'modo'
]

In [21]:
stopwords_strong = [word for word in stopwords_strong if word not in protectedWords]
docs_and_stopwords = Counter([word for word in docWords if word in stopwords_strong])

In [22]:
docs_and_stopwords.most_common(100)

[('u', 23849),
 ('puede', 20530),
 ('si', 16871),
 ('sino', 12907),
 ('mismo', 12535),
 ('i', 10742),
 ('decir', 9864),
 ('manera', 8405),
 ('parte', 8370),
 ('sólo', 7891),
 ('dos', 7645),
 ('tal', 7324),
 ('hace', 6968),
 ('posible', 6948),
 ('pues', 6780),
 ('así', 6768),
 ('misma', 6477),
 ('tiempo', 6370),
 ('lugar', 6328),
 ('solo', 6229),
 ('embargo', 6179),
 ('debe', 6164),
 ('hecho', 6036),
 ('vez', 5925),
 ('ejemplo', 5708),
 ('verdad', 5646),
 ('entonces', 5538),
 ('parece', 5156),
 ('aquí', 5151),
 ('toda', 4949),
 ('ello', 4911),
 ('según', 4852),
 ('cómo', 4771),
 ('siempre', 4722),
 ('cosas', 4597),
 ('cada', 4515),
 ('poder', 4501),
 ('respecto', 4312),
 ('pueden', 4281),
 ('hacer', 4219),
 ('podría', 4204),
 ('general', 4174),
 ('da', 4164),
 ('menos', 4066),
 ('partir', 4046),
 ('saber', 4046),
 ('trata', 3982),
 ('primera', 3926),
 ('h', 3904),
 ('primer', 3887),
 ('cuanto', 3823),
 ('cierto', 3644),
 ('propia', 3568),
 ('trabajo', 3560),
 ('aunque', 3427),
 ('cuenta

Additionally, we have added words that we observed were incorrectly lemmatized. We will pass the list of protected words to the lemmatizer later on and we will skip these protected words.

In [23]:
protectedWords += [
    'parte',
    'posible',
    'lugar',
    'hecho',
    'poder',
    'verdad',
    'cosas',
    'general',
    'fin',
    'trabajo',
    'cierto',
    'uso',
    'dado',
    'diferentes',
    'verdadero',
    'verdadera',
    'existe',
    'valor',
    'realizar',
    'existen',
    'conocer',
    'diferente',
    'idea',
    'caso',
    'consciencia',
    'conciencia',
    'objeto',
    'forma',
    'obra',
    'persona',
    'sujeto',
    'primer',
    'primera',
    'primero',
    'descartes',
    'libre',
    'libres',
    'regla',
    'signo',
    'liberté',
    'potencia',
    'cosa',
    'nombre',
    'enunciado',
    'profundo',
    'moneda',
    'minuto',
    'madera',
    'indicio',
    'industria',  
    'espejo',
    'escolio',
    'era',
    'prototipo',
    'discurso',
    'escritura'
    ]

protectedWords = list(set(protectedWords))

After a couple of times to make the process less complex, once we are sure of a set of words, we can eliminate those from the list of document words and go back and repeat the process a couple more times.

In [24]:
stopwordsToRemove = [word[0] for word in docs_and_stopwords.most_common(100)]
docWords = [word for word in docWords if word not in stopwordsToRemove]

### Removing stopwords in English

Given that most of the articles have abstracts in English, some of the usual stopwords in English are appearing frequently in our documents. Thus, we will append the NLTK-generated list of English stopwords. We will also use one for Portuguese, which we sometimes get as well.

In [25]:
englishStopwords = nltk_stopwords.words("english")
portugueseStopwords = nltk_stopwords.words("portuguese")

TODO: should we be filtering these just like we filter the Spanish ones?

### Other stopwords custom to our corpus

There are some other stopwords that we would like to include, but that have not been taken into account in the previous processes. These are:

In [26]:
customStopwords = [
    "cf",
    "no obstante",
    "sin embargo",
    "por ejemplo",
    "es decir",
    "ak",
    "krv",
    "tha",
    "press",
    "university",
    "est",
    "non",
    "par",
    "per",
    "tod",
    "ell",
    "cua",
    "alg",
    "segú",
    "chic",
    "thi",
    "cad",
    "hac",
    "ca",
    "pue",
    "cambridge",
    "would"
]

Once we have a robust set of words we can save both the final stopword list and the protected words list.

In [27]:
stopwords_final = list(set(
    stopwords_weak + stopwords_strong + englishStopwords + portugueseStopwords +  customStopwords
))
with open('wordlists/stopwords.txt', 'w') as fp:
    fp.write('\n'.join(stopwords_final))

with open('wordlists/protectedWords.txt', 'w') as fp:
    fp.write('\n'.join(protectedWords))

# Final replacements and edits
PDF correction is not perfect and we observe some artifacts left in the LDA. A hotfix is to do those replacements manually for now and check whether we can improve on this process in the future.

In [28]:
manualReplacements = {
    'kan': 'kant',
    'Kan': 'Kant',
    'entr': 'entre',
    'otr': 'otro',
    'mism': 'mismo',
    'wit': 'with',
    'tien': 'tiene',
    'maner': 'manera',
    'objet': 'objeto',
    'Hege': 'Hegel',
    'mund': 'mundo',
    'sistem': 'sistema',
    'obr': 'obra',
    'histori': 'historia',
    'pode': 'poder',
    'deci': 'decir',
    'bie': 'bien',
    'entonce': 'entonces',
    'verda': 'verdad',
    'deb': 'deber',
    'tant': 'tanto',
    'mora': 'moral',
    'form': 'forma',
    'Hum': 'Hume',
    'ide': 'idea',
    'mod': 'modo',
    'hech': 'hecho',
    'vid': 'vida',
    'relativ': 'relativo',
    'negativ': 'negativo',
    'mínim': 'mínimo',
    'implícit': 'implícito',
    'gegenealógic': 'genealógico',
    'explícit': 'explícito',
    'disput': 'disputa',
    'liberta': 'libertad',
    'polític': 'política'
    
}

In [29]:
for doc in corpusList:
    for word, replacement in manualReplacements.items():
        doc.cleanText = re.sub(r'\b' + word + r'\b', replacement, doc.cleanText)

In [30]:
if not os.path.exists('../data/corpus'):
    os.mkdir('../data/corpus')
saveCorpus('../data/corpus', corpusList)